# SHARKS ATTACKS

# Data import and preview of the data

In [77]:
import numpy as np
import pandas as pd

data = pd.read_csv('GSAF5.csv',  encoding='mac-roman')
display(data.head())
display(data.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


(5992, 24)

# 0. Cleaning columns names

Como primer paso, se eliminan los espacios al principio y final de los nombres de las columnas para que las mismas puedan ser referenciadas sin errores.
A continuación, se aplica una fórmula para eliminar los espacios al principio y final de cada una de las celdas.

In [78]:
columns = data.columns
col_names_clean = [name.strip() for name in columns]
data.columns = col_names_clean
data = data.applymap(lambda x: x.strip() if type(x) is str else x)

#    1. Missing Values analysis

In [79]:
null_cols = data.isnull().sum()
display(null_cols[null_cols > 0])
display(null_cols[null_cols == 0])

numeric_columns = [col for col in data._get_numeric_data()]
display(numeric_columns)

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

Case Number       0
Date              0
Year              0
Type              0
pdf               0
Case Number.1     0
Case Number.2     0
original order    0
dtype: int64

['Year', 'original order']

According to the results, it is fair to conclude that columns "Unnamed 22" and "Unnamed 23" can be dropped.
Además, los únicos campos puramente numéricos, Año y 'original order', no contienen nulos. El campo edad, que debería ser numérico, no lo es porque algunos registros contienen texto (que se limpiará o traducirá en pasos posteriores).

In [80]:
drop_cols = ['Unnamed: 22', 'Unnamed: 23']

Se observa también que el valor de los campos Case Number.1 y Case Number.2 es igual que el de Case Number. Lo verificamos con un bucle:

In [81]:
for i, row_value in data['Case Number'].iteritems():
    if row_value != data['Case Number.1'][i] and row_value != data['Case Number.2'][i]:
        print(i, 'Mismatch founded', row_value, data['Case Number.1'][i], data['Case Number.2'][i])

169 Mismatch founded 2015.07-10 2015.07.10 2015.07.10
4719 Mismatch founded 1934.01.08.R 1934.02.08.R 1934.02.08.R


Para mantener la columna 'Case Number', corregimos las discrepancias y eliminamos las columnas Case Number 1 y 2

In [82]:
data.loc[169, 'Case Number'] = data['Case Number.1'][169]
data.loc[4719, 'Case Number'] = data['Case Number.1'][4719]
drop_cols += ['Case Number.1', 'Case Number.2']

También a priori parece que el contenido de las columnas 'href formula' y 'href' es el mismo. Comprobamos:

In [83]:
mismatch_href_count = 0
for i, row_value in data['href'].iteritems():
    if row_value != data['href formula'][i]:
        print(i, 'Mismatch founded', row_value, data['href formula'][i])
        mismatch_href_count += 1
display(mismatch_href_count)

20 Mismatch founded http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.29-Spain.pdf http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.29-Spain.pdf
27 Mismatch founded http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.23-Cutbirth.pdf http://sharkattackfile.net/spreadsheets/pdf_directory/2016.07.23.a-Cutbirth.pdf
61 Mismatch founded http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2016.05.21.a-Girl.pdf http://sharkattackfile.net/spreadsheets/pdf_directory/2016.05.21.a-Girl.pdf
107 Mismatch founded http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2015.12.21.a-Brazil.pdf http://sharkattackfile.net/spreadsheets/pdf_directory/2015.12.21.a-Brazil.pdf
114 Mismatch founded http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2015.11.15.a

53

Se puede ver que para 53 filas, los contenidos de las columnas no coinciden. Sin embargo, dado que este valor no llega al 1% de los registros, lo consideramos despreciable. Además, el contenido del campo no nos aporta información para nuestro análisis a alto nivel. Por tanto, eliminamos también la columna 'href'.

In [84]:
drop_cols += ['href']
data = data.drop(drop_cols, axis=1)

In [85]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


It can be observed that the name in some of the rows of the dataset is male and female. Así que se puede utilizar este valor para tratar de rellenar aquellos valores null del campo Sex. Además se comprobará si hay alguna discrepancia de sexo entre el campo namy y el de female.

In [6]:
for i in range(len(data['Sex'])):
    if data.loc[i, 'Name'] == 'male' and data.loc[i, 'Sex'] != 'M' or data.loc[i, 'Name'] == 'female' and data.loc[i, 'Sex'] != 'F':
        print(i, data.loc[i, 'Name'], data.loc[i, 'Sex'], data.loc[i, 'Injury'])

397 female M Lacerations to right foot
405 female M Arm bitten by captive shark PROVOKED INCIDENT
1046 female M Cuts & punctures to right foot
2000 male nan 2 small lacerations to bottom of foot
2824 male nan Probable drowning & scavenging
3068 male nan FATAL, thigh bitten
4112 male nan Human remains found in shark
4240 male nan Harpooned shark bit his forehead PROVOKED INCIDENT
5018 male nan FATAL
5603 female M Partial human remains found in 13' shark
5779 male nan Lacerations to leg  PROVOKED INCIDENT
5825 male nan FATAL
5865 male nan Foot severed


No se ha encontrado un alto número de discrepancias entre, pero para los resultados conseguidos completaremos los valores null del campo Sex.

In [7]:
data.loc[2000, 'Sex'] = 'M'
data.loc[2824, 'Sex'] = 'M'
data.loc[3068, 'Sex'] = 'M'
data.loc[4112, 'Sex'] = 'M'
data.loc[4240, 'Sex'] = 'M'
data.loc[5018, 'Sex'] = 'M'
data.loc[5779, 'Sex'] = 'M'
data.loc[5825, 'Sex'] = 'M'
data.loc[5865, 'Sex'] = 'M'

# 2. Incorrect data identification

La primera idea que nos surge para comprobar la corrección de los datos es verificar si las ciudades que se lista en la columna Area pertenecen realmente al país en la columna Country. Para ello recurrimos a una librería externa, Geopy, pero nos encontramos el problema de que la ejecución de la verificación es muy lenta y de que los nombres de los países devueltos están en idioma y/o alfabeto local, con lo cual no se pueden comparar con el país mostrado en el dataset. Además, para algunas ciudades el servidor no devuelve resultad.

Se incluye el código a continuación:

In [8]:
#from geopy.geocoders import Nominatim
#geolocator = Nominatim(timeout=3)
#for i in range(len(data['Area'])):
    #location = geolocator.geocode(data.loc[i, 'Area'])
    #if location:
    #   location = location.raw['display_name'].split(',')
    #country = location['display_name'].split(',')
    #if location[1].lower().strip() == data.loc[i, 'Country'].lower().strip():
         #print(i, location[1].lower(), data.loc[i, 'Country'].lower())

Como segundo paso a la comprobación de datos incorrectos, se podría verificar que el sexo de las víctimas coincide con el valor del campo nombre cuando en este último se incluye male o female. Sin embargo, esta comprobación ya se hizo en el punto 1, comprobación de nulos.

Se comprobará que todos los registros para los que el campo Injury indica 'FATAL', la columna 'Fatal (Y/N)' indica Y.

In [9]:
for i in range(len(data['Injury'])):
    if data.loc[i, 'Injury'] == 'FATAL' and data.loc[i, 'Fatal (Y/N)'] != 'Y':
        print(i, data.loc[i, 'Injury'], data.loc[i, 'Fatal (Y/N)'])

2449 FATAL nan
5718 FATAL nan


A partir de las dos discrepancias anteriores, rellenamos el valor null:

In [10]:
data.loc[2449, 'Fatal (Y/N)'] = 'Y'
data.loc[5718, 'Fatal (Y/N)'] = 'Y'

# 3. Data type correction

Para empezar se comprueba que en la columna 'Date', muchos de los valores, a pesar de tener un formato similar al de una fecha en realidad no lo son, y otros incluyen caracteres de texto como 'Reported'. Para obtener las fechas exactas expresadas en formato fecha, se recurre a la columna 'Case Number'.

In [11]:
import datetime

counter = 0
for i in range(len(data['Case Number'])):
    counter += 1
    x = data.loc[i, 'Case Number']
    #print(counter, x)
    try:
        year = int(x[:4])
        month = int(x[5:7]) if int(x[5:7]) != 0 else 1
        day = int(x[8:10]) if int(x[8:10]) != 0 else 1
        #print(counter, x, '.....', year, month, day)
        date_value = datetime.date(year, month, day)
        data.loc[i, 'Date'] = date_value
    except:
        pass


In [18]:
data_counter_not_valid_date = 0
for x in data['Date']:
    if type(x) is not datetime.date:
        data_counter_not_valid_date += 1
print('Percentage of non valid dates: ', round(data_counter_not_valid_date/len(data['Date']) * 100, 3), '%')

Percentage of non valid dates:  2.069 %


Gracias al código anterior, hemos conseguido transformar a objetos de fechas gran parte de los registros, lo que nos permitirá ubicarlos de forma correcta en un eje temporal. 
Comprobamos también que sólo un pequeño porcentaje de los registros (2.07%) no ha sido posible de ubicar de forma correcta. Estos registros, mediante análisis cualitativo, puede comprobarse que típicamente se corresponden con fechas muy lejanas.

Un vez que hemos rellenado correctamente el campo 'Date', se puede verificar que el año se corresponde con el valor del campo 'Year' mediante el código siguiente:

In [26]:
data_counter_error_years = 0

for i, row_value in data['Year'].iteritems():
    try:
        if data['Date'][i].year != row_value:
            print(i, 'Year mismatch', '     Valor del campo "Date": ', data['Date'][i], '     Valor del campo "Year": ', row_value, '    Valor de Case Number: ',data['Case Number'][i])
            data_counter_error_years += 1
    except:
        pass
    
print('Percentage of mistmatches: ', round(data_counter_error_years/len(data['Year']) * 100, 3), '%')

Percentage of mistmatches:  0.0 %


Se comprueba que para algunos registros, los valores de ambos campos no coinciden, por lo que procedemos a corregir el campo 'Year':

In [25]:
data.loc[2449, 'Year'] = 1989
data.loc[3662, 'Year'] = 1961
data.loc[4312, 'Year'] = 1948
data.loc[4983, 'Year'] = 1923
data.loc[5043, 'Year'] = 1900

A continuación se hará que todos los campos no númericos (todos menos 'Date', 'Year', 'original order' y 'Age'] sean strings.

In [44]:
text_columns = ['Country', 'Area', 'Location', 'Activity', 'Name', 'Sex', 'Injury', 'Fatal (Y/N)', 'Time', 'Species', 'Investigator or Source', 'pdf', 'href formula', 'href']
data[text_columns] = data[text_columns].astype('str')
data['Country'] = data['Country'].astype(str)
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

# 3. Values normalization

Para facilitar las tareas de análisis finales, se tratará de normalizar los valores de cada columna en la medida de lo posible eliminando espacios y caracteres especiales.

COLUMNA 'Activity':

In [15]:
data['Type'].unique()

array(['Unprovoked', 'Boat', 'Provoked', 'Invalid', 'Sea Disaster',
       'Boating'], dtype=object)

Los resultados Boat y Boating paracen ser la misma categoría, por lo que normalizaremos los valores renombrado los 'boating' como 'boat'.

In [27]:
data['Type'] = data['Type'].str.replace('Boating', 'Boat')
data['Type'].unique()

array(['Unprovoked', 'Boat', 'Provoked', 'Invalid', 'Sea Disaster'],
      dtype=object)

COLUMNA 'Country':

In [59]:
display(np.sort(data['Country'].unique()))
display(data.groupby(['Country']).size())
#display(data.groupby('Country').count())

array(['ADMIRALTY ISLANDS', 'ALGERIA', 'AMERICAN SAMOA',
       'ANDAMAN / NICOBAR ISLANDAS', 'ANGOLA', 'ANTIGUA', 'ARGENTINA',
       'ARUBA', 'ASIA?', 'ATLANTIC OCEAN', 'AUSTRALIA', 'AZORES',
       'BAHAMAS', 'BAHREIN', 'BANGLADESH', 'BARBADOS', 'BAY OF BENGAL',
       'BELIZE', 'BERMUDA', 'BRAZIL', 'BRITISH ISLES',
       'BRITISH NEW GUINEA', 'BRITISH VIRGIN ISLANDS',
       'BRITISH WEST INDIES', 'BURMA', 'Between PORTUGAL & INDIA',
       'CANADA', 'CAPE VERDE', 'CARIBBEAN SEA', 'CAYMAN ISLANDS',
       'CENTRAL PACIFIC', 'CEYLON (SRI LANKA)', 'CHILE', 'CHINA',
       'COLUMBIA', 'COOK ISLANDS', 'COSTA RICA', 'CRETE', 'CROATIA',
       'CUBA', 'CURACAO', 'CYPRUS', 'Coast of AFRICA', 'DIEGO GARCIA',
       'DJIBOUTI', 'DOMINICAN REPUBLIC', 'ECUADOR', 'EGYPT',
       'EGYPT / ISRAEL', 'EL SALVADOR', 'ENGLAND',
       'EQUATORIAL GUINEA / CAMEROON', 'FALKLAND ISLANDS',
       'FEDERATED STATES OF MICRONESIA', 'FIJI', 'FRANCE',
       'FRENCH POLYNESIA', 'Fiji', 'GABON', 'GEORGIA', 

Country
ADMIRALTY ISLANDS                1
ALGERIA                          1
AMERICAN SAMOA                   3
ANDAMAN / NICOBAR ISLANDAS       1
ANGOLA                           1
ANTIGUA                          1
ARGENTINA                        1
ARUBA                            1
ASIA?                            1
ATLANTIC OCEAN                  16
AUSTRALIA                     1279
AZORES                           5
BAHAMAS                         98
BAHREIN                          1
BANGLADESH                       1
BARBADOS                         5
BAY OF BENGAL                    1
BELIZE                           3
BERMUDA                         16
BRAZIL                         102
BRITISH ISLES                    1
BRITISH NEW GUINEA               1
BRITISH VIRGIN ISLANDS           1
BRITISH WEST INDIES              1
BURMA                            4
Between PORTUGAL & INDIA         1
CANADA                          10
CAPE VERDE                       3
CARIBBEAN SE

Se observa que no se repiten los valores de los países, aunque hay uno de ellos que no es un país "Diego García" y la mayoría de los países aparecen en un único registro.

In [63]:
for i in range(len(data['Country'])):
    if data['Country'][i] == 'DIEGO GARCIA':
        print(i, data.loc[i, 'original order'])

432 5561
